In [2]:
import numpy as np 
import msprime
from scipy import linalg, stats
import utils

np.random.seed(42)

## Setup: read in data and simulate data

In [5]:
# Real data from 1000 genomes project
G = np.load("../processed_data/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.npz")['arr_0']
svd_data = np.load("../processed_data/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes)_SVD.npz")

U, S, Vt = svd_data["U"], svd_data["S"], svd_data["Vt"]

In [6]:
# We simulate data of various sizes, sequence lengths, and sparsities
sample_sizes = [10**x for x in range(2, 5)]
sequence_lengths = [10**x for x in range(2, 8)]
mutation_rates = [10**(-x) for x in range(1, 10)] #lower mutation rate = sparser
recombination_rates = [10**(-x) for x in range(1, 10)] #lower recombination rate = sparser

G_simulated = {}

for n in sample_sizes:
    for length in sequence_lengths:
        for mut_rate in mutation_rates:
            for recomb_rate in recombination_rates:
                G = utils.simulate_diploid_genotypes(n, length, mut_rate, recomb_rate)
                G_simulated[(n, length, mut_rate, recomb_rate)] = G

In [ ]:
# Save simulated data
data_dir = "../processed_data"
for (params, G) in G_simulated.items():
    file_name = f"{params[0]}_{params[1]}_{params[2]}_{params[3]}.genotypes.npz"
    np.savez_compressed(f"{data_dir}/{file_name}", G)

## Darnell et al: ARSVD

Paper: https://www.jmlr.org/papers/volume18/15-143/15-143.pdf 

Implementation: https://github.com/gdarnell/arsvd/blob/master/dimension_reduction.py 


In [3]:
def rsvd(X, dstar, power_iters=2, delta=10):
	""" Perform rsvd algorithm on input matrix.
		Method must be supplied dstar.
		Returns truncated svd (U,S,V).
	Parameters
	----------
	X : int matrix
    	Matrix of n x m integers, where m <= n. If n < m,
    	matrix will be transposed to enforce m <= n.
   	dstar : int
   		The latent (underlying) matrix rank that will be
   		used to truncate the larger dimension (m).
   	power_iters : int
   		default: 2
   		Number of power iterations used (random matrix multiplications)
   	delta : int
   		default: 10
   		oversampling parameter (to improve numerical stability)
    Returns
	-------
	int matrix
    	Matrix of left singular vectors.
    int matrix
    	Matrix of singular values.
    int matrix
    	Matrix of right singular vectors.
    """
	transpose = False 
	if X.shape[0] < X.shape[1]:
		X = X.T 
		transpose = True 

	if power_iters < 1:
		power_iters = 1

	# follows manuscript notation as closely as possible
	P = np.random.randn(X.shape[0],dstar+delta)	
	for i in range(power_iters):
		P = np.dot(X.T,P)
		P = np.dot(X,P)
	Q,R = np.linalg.qr(P)
	B = np.dot(Q.T,X)
	U,S,V = linalg.svd(B)
	U = np.dot(Q, U)

	# Remove extra dimensionality incurred by delta
	U = U[:, 0:dstar]
	S = S[0:dstar]

	return V.T, S, U.T if transpose else U, S, V


def stabilityMeasure(X, d_max, B=5, power_iters=2):
	""" Calculate stability of 
	Parameters
	----------
	X : int matrix
		input matrix to determine rank of
	d_max : int
		upper bound rank to estimate
	B : int
		default: 5
		number of projections to correlate
	power_iters : int
		default: 2
   		Number of power iterations used (random matrix multiplications)
	Returns
	-------
	int
		Latent (lower-dimensional) matrix rank
	"""
	singular_basis = np.zeros((B,X.shape[0],d_max))
	# calculate singular basis under multiple projections
	for i in range(B):
		U = rsvd(X,d_max)[0]
		singular_basis[i,:,:] = U[:,0:d_max]

	# calculate score for each singular vector
	stability_vec = np.zeros((d_max))
	for k in range(d_max):
		stability = 0
		for i in range(0,B-1):
			for j in range(i+1,B):
				corr = stats.spearmanr(singular_basis[i,:,k],singular_basis[j,:,k])[0]
				stability = stability + abs(corr)
		N = B*(B-1)/2
		stability = stability/N
		stability_vec[k] = stability

	# wilcoxon rank-sum test p-values
	p_vals = np.zeros(d_max-2)
	for k in range(2,d_max):
		p_vals[k-2] = stats.ranksums(stability_vec[0:k-1],stability_vec[k-1:d_max])[1]

	dstar = np.argmin(p_vals)
	
	return dstar

## Saibaba et al: GSVD

Paper: https://www.osti.gov/servlets/purl/1769894 

In [ ]:
def gsvd():
    